In [1]:
# Disable Python bytecode cache to prevent stale .pyc files
import sys
import os
sys.dont_write_bytecode = True
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
print("Python bytecode cache disabled")


Python bytecode cache disabled


Setup: Disable Python Bytecode Cache

This cell prevents Python from creating .pyc cache files that can cause stale code to be loaded even after source files are updated.


# Snow Elastic Modulus Calculations - Bergfeld et al. (2023) Method

This notebook demonstrates the application of the Bergfeld et al. (2023) elastic modulus parameterization to snowpit data. The Bergfeld method calculates elastic modulus from snow density using a power-law relationship optimized from Propagation Saw Test (PST) data.

The analysis uses the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from uncertainties import ufloat
import warnings


# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


Parse Snowpit Files


In [4]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Apply Bergfeld parameterization using density and associated uncertainty from:
1. Direct Measurement
2. Kim_geldsetzer

In [5]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []
density_info = []  # New list for density observations

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_depth_top = layer.depth_top[0] if layer.depth_top else None
        layer_thickness = layer.thickness[0] if layer.thickness else None
        
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer_depth_top,      # Add for merging (scalar)
            'thickness': layer_thickness,      # Add for merging (scalar)
        }

        # Add kim_geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer')
            try: 
                # Calculate density using kim_geldsetzer method - only if we have valid inputs
                if layer.hardness and layer_dict['kim_geldsetzer_grain_form']:
                    density_ufloat = density.calculate_density( 
                        method='kim_geldsetzer',
                        hand_hardness=layer.hardness,
                        grain_form=layer_dict['kim_geldsetzer_grain_form']
                    )
                    layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                    layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
                else:
                    layer_dict['density_kim_geldsetzer'] = None
                    layer_dict['density_kim_geldsetzer_uncertainty'] = None
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['kim_geldsetzer_grain_form'] = None
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        layer_info.append(layer_dict)

    # Collect density observations separately
    for density_obs in pit.snow_profile.density_profile:
        # Extract scalar density value
        density_value = density_obs.density
        if hasattr(density_value, '__len__') and len(density_value) > 0:
            density_value = density_value[0]  # Take first element if it's an array
        if hasattr(density_value, 'nominal_value'):
            density_value = density_value.nominal_value  # Extract nominal value if it's a ufloat
        
        obs_depth_top = density_obs.depth_top[0] if density_obs.depth_top else None
        obs_thickness = density_obs.thickness[0] if density_obs.thickness else None
        
        density_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_top': obs_depth_top,
            'thickness': obs_thickness,
            'density_measured': float(density_value) if density_value is not None else None
        }
        density_info.append(density_dict)

# Create dataframes
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)
density_df = pd.DataFrame(density_info)

# Merge density_measured into layer_df using pandas merge (efficient matching)
layer_df = layer_df.merge(
    density_df[['pit_id', 'depth_top', 'thickness', 'density_measured']],
    on=['pit_id', 'depth_top', 'thickness'],
    how='left'
).drop(columns=['depth_top', 'thickness'])  # Drop temporary columns after merge


Apply Bergfeld Elastic Modulus Parameterization

Now we'll apply the Bergfeld et al. (2023) method to calculate elastic modulus for all layers that have density values.


In [6]:
# Apply Bergfeld Elastic Modulus Parameterization to both density methods

# Initialize results storage
results = {
    'kim_geldsetzer': {
        'elastic_modulus_values': [],
        'successful_count': 0
    },
    'measured': {
        'elastic_modulus_values': [],
        'successful_count': 0
    }
}

# Process kim_geldsetzer density method (with uncertainty)
print("Processing kim_geldsetzer density method...")
for idx, row in layer_df.iterrows():
    if pd.notna(row['density_kim_geldsetzer']) and pd.notna(row['density_kim_geldsetzer_uncertainty']):
        # Create ufloat with density and uncertainty
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        
        # Calculate elastic modulus using Bergfeld method
        try:
            E_modulus = elastic_modulus.calculate_elastic_modulus(method='bergfeld', density=density_ufloat)
            
            # Check if result is valid (not NaN)
            if not np.isnan(E_modulus.nominal_value):
                results['kim_geldsetzer']['elastic_modulus_values'].append(E_modulus)
                results['kim_geldsetzer']['successful_count'] += 1
                
                # Store results back in dataframe
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = E_modulus.nominal_value
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = E_modulus.std_dev
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_relative_uncertainty'] = E_modulus.std_dev / E_modulus.nominal_value
            else:
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = np.nan
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = np.nan
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_relative_uncertainty'] = np.nan
                
        except Exception as e:
            layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = np.nan
            layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = np.nan
            layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_relative_uncertainty'] = np.nan
# Process measured density method (without uncertainty)
print("Processing measured density method...")
for idx, row in layer_df.iterrows():
    density_measured = row['density_measured']
    
    # Handle the case where density_measured might be an array or have multiple values
    if density_measured is not None:
        # Convert to scalar if it's an array
        if hasattr(density_measured, '__len__') and not isinstance(density_measured, str):
            if len(density_measured) > 0:
                density_measured = density_measured[0]
            else:
                density_measured = None
        
        # Check if we have a valid scalar value
        if density_measured is not None and pd.notna(density_measured):
            # Create ufloat with density but no uncertainty (std_dev = 0)
            density_ufloat = ufloat(float(density_measured), 0.0)
        
            # Calculate elastic modulus using Bergfeld method
            try:
                E_modulus = elastic_modulus.calculate_elastic_modulus(method='bergfeld', density=density_ufloat)
                
                # Check if result is valid (not NaN)
                if not np.isnan(E_modulus.nominal_value):
                    results['measured']['elastic_modulus_values'].append(E_modulus)
                    results['measured']['successful_count'] += 1
                    
                    # Store results back in dataframe
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = E_modulus.nominal_value
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = E_modulus.std_dev
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured_relative_uncertainty'] = E_modulus.std_dev / E_modulus.nominal_value
                else:
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = np.nan
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = np.nan
                    
            except Exception as e:
                layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = np.nan
                layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = np.nan


Processing kim_geldsetzer density method...
Processing measured density method...


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [7]:

# Print summary statistics
print("\n=== BERGFELD ELASTIC MODULUS CALCULATION RESULTS ===")
print("Kim-Geldsetzer method:")
print(f"  - Total layers with kim_geldsetzer density: {layer_df['density_kim_geldsetzer'].notna().sum()}")
print(f"  - Successful calculations: {results['kim_geldsetzer']['successful_count']} ({(results['kim_geldsetzer']['successful_count'] / layer_df['density_kim_geldsetzer'].notna().sum()) * 100:.2f}%)")
print(f"  - Average relative uncertainty: {layer_df['elastic_modulus_bergfeld_kim_geldsetzer_relative_uncertainty'].mean():.2f}%")

print("\nMeasured density method:")
print(f"  - Total layers with measured density: {layer_df['density_measured'].notna().sum()}")
print(f"  - Successful calculations: {results['measured']['successful_count']} ({(results['measured']['successful_count'] / layer_df['density_measured'].notna().sum()) * 100:.2f}%)")
print(f"  - Average relative uncertainty: {layer_df['elastic_modulus_bergfeld_measured_relative_uncertainty'].mean():.2f}%")

# Save updated dataframe
layer_df.to_csv('layer_df.csv', index=False)
print("\nUpdated layer data saved to layer_df.csv")



=== BERGFELD ELASTIC MODULUS CALCULATION RESULTS ===
Kim-Geldsetzer method:
  - Total layers with kim_geldsetzer density: 235522
  - Successful calculations: 207373 (88.05%)
  - Average relative uncertainty: 0.80%

Measured density method:
  - Total layers with measured density: 10468
  - Successful calculations: 6825 (65.20%)
  - Average relative uncertainty: 0.24%

Updated layer data saved to layer_df.csv
